<a href="https://colab.research.google.com/github/Essie-Wabomba-dmd/Call-Centre-Trends-Analysis-with-Power-BI/blob/main/Interacting_with_CLIP_(Li)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

Torch version: 2.1.0+cu121


In [2]:
!pip install clip torch

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=2f184ff7ae046c84f9df7dd01e3f2ed67d0d2c584750d5509352dd16ab43a38e
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [3]:
!pip install tqdm

In [4]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image

In [5]:
from transformers import CLIPProcessor, CLIPModel
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [6]:
os.environ["HUGGINGFACE_TOKEN"] = "your Token"

In [7]:
# Load CLIP model and processor
clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

## Image Preprocessing
Normalize the pixel intensity using the dataset mean and standard deviation the resize the input images and center-crop them to conform with the image resolution that the model expects

In [8]:
# Define transformation for images
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [9]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import skimage

In [10]:
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [11]:
# Custom dataset class
original_images = []
images = []
texts = []
class CustomDataset(Dataset):
    def __init__(self, csv_file, image_folder, clip_model, clip_processor, transform=None):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.transform = transform
        self.clip_model = clip_model
        self.clip_processor = clip_processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        img_path = os.path.join(self.image_folder,  f"{img_name}.png")
        image = Image.open(img_path).convert("RGB")
        description = self.data.iloc[idx, 2]
        price = self.data.iloc[idx, 3]


        # Preprocess image
        if self.transform:
            image = self.transform(image)

        # Extract image features using CLIP model
        inputs = self.clip_processor(images=image, return_tensors="pt", padding=True)
        with torch.no_grad():
            image_features = self.clip_model.get_image_features(**inputs)

        # Extract text features using CLIP model
        inputs = self.clip_processor(text=[description], return_tensors="pt", padding=True)
        with torch.no_grad():
            text_features = self.clip_model.get_text_features(**inputs)

        sample = {'image': image, 'description': description, 'price': price,
                  'image_features': image_features, 'text_features': text_features}
        return sample